## <center>Лабораторная работа № 6 'Прогноз успеха фильмов по обзорам'<center>

### <center>Выполнила студентка 3 курса группы БФИ2001 Калмыкова Дарья<center>

### Цель
Прогноз успеха фильмов по обзорам (Predict Sentiment From Movie Reviews)

### Задачи
* Ознакомиться с задачей классификации
* Изучить способы представления текста для передачи в ИНС
* Достигнуть точность прогноза не менее 95%

### Требования
1. Построить и обучить нейронную сеть для обработки текста
2. Исследовать результаты при различном размере вектора представления текста
3. Написать функцию, которая позволяет ввести пользовательский текст (в отчете 
привести пример работы сети на пользовательском тексте

In [35]:
import numpy as np
import torch
import re
import spacy

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv2D
from keras.datasets import imdb
from keras.regularizers import l2

import tensorflow as tf
from tensorflow.keras.initializers import glorot_uniform
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

In [4]:
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))

Categories: [0 1]
Number of unique words: 9998


In [5]:
length = [len(i) for i in data]

In [6]:
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Average Review length: 234.75892
Standard Deviation: 173


In [7]:
print("Label:", targets[0])
print(data[0])

Label: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [8]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()])
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] )
print(decoded)

# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

In [9]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

data = vectorize(data)
targets = np.array(targets).astype("float32")

In [10]:
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

In [11]:
model = Sequential()

model.add(Dense(50, activation = "relu", input_shape=(10000, )))

model.add(Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(50, activation = "relu"))

model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(Dense(50, activation = "relu"))

model.add(Dense(1, activation = "sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                500050    
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 505,201
Trainable params: 505,201
Non-trai

In [12]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", 
              metrics = ["accuracy"])

In [14]:
results = model.fit(train_x, train_y, epochs= 2, batch_size = 500, 
                    validation_data = (test_x, test_y))

Epoch 1/2
80/80 [==============================] - 3s 26ms/step - loss: 0.4009 - accuracy: 0.8230 - val_loss: 0.2778 - val_accuracy: 0.8850
Epoch 2/2
80/80 [==============================] - 2s 20ms/step - loss: 0.2151 - accuracy: 0.9181 - val_loss: 0.2615 - val_accuracy: 0.8953


In [15]:
print(np.mean(results.history["val_accuracy"]))

0.8901499807834625


### Достигнем максимально возможной точности прогноза на валидационных данных (без применения Transfer Learning, Ensemble Modeling, Word Embeddings, etc)

In [18]:
model_exp = Sequential()

model_exp.add(BatchNormalization(synchronized=True))

model_exp.add(Dense(60, activation = "relu", input_shape=(10000, ),
                       kernel_initializer=glorot_uniform()))
model_exp.add(Dense(150, activation='relu',
                   kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model_exp.add(Dropout(0.3))

model_exp.add(BatchNormalization(synchronized=True))

model_exp.add(Dense(90, activation='relu',
                   kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model_exp.add(Dropout(0.2))

model_exp.add(BatchNormalization(synchronized=True))

model_exp.add(Dense(40, activation='relu',
                   kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model_exp.add(Dense(1, activation='sigmoid'))

In [19]:
model_exp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_exp.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=3, batch_size=350, verbose=1)
print(np.mean(results.history["val_accuracy"]))

Epoch 1/3
115/115 [==============================] - 7s 41ms/step - loss: 0.4074 - accuracy: 0.8050 - val_loss: 0.3444 - val_accuracy: 0.8749
Epoch 2/3
115/115 [==============================] - 4s 37ms/step - loss: 0.1806 - accuracy: 0.9317 - val_loss: 0.2895 - val_accuracy: 0.8834
Epoch 3/3
115/115 [==============================] - 4s 38ms/step - loss: 0.1010 - accuracy: 0.9645 - val_loss: 0.3260 - val_accuracy: 0.8822
0.8901499807834625


### Функция, которая позволяет ввести пользовательский текст

In [23]:
def preproc(text):
    text_punct = re.sub(r"[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]", ' ', text)
    text_numbers = re.sub(r"[0-9]", ' ', text_punct)
    
    text_token = word_tokenize(text_numbers, language='english')
    stop = set(stopwords.words('english'))
    text_preproc = [word for word in text_token if word not in stop]
    
#     doc = nlp(text_preproc)
#     text_lemmed = [word.lemma_ for word in doc]

    return text_preproc

In [24]:
def get_index(words_prep):
    res = []
    for i in range(len(words_prep)):
        word_index = imdb.get_word_index()
        word = words_prep[i]
        if word_index[word] < 10000:
            res.append(word_index[word])
    return res

In [25]:
def get_vec(words_idx):
    n = 10000
    words_ready = [0]*n
    for i in range(n):
        if i in words_idx and i < n:
            words_ready[i] = 1
    return np.asarray(words_ready).astype('float64')

#### Positive review

In [26]:
text_string = str(input('Input your movie review: '))
words = preproc(text_string.lower())
words_prep = words.copy()
words_idx = get_index(words_prep)

Input your movie review: this movie was so well written and the casting was awesome that i must say it's great. Ben Affleck is quite good, and Rosamund Pike is great. The whole plot keeps you guessing and on the edge of your seat. Overall, a great movie. I highly recommend! Great to watch with significant other, not so much for families. movie well written casting awesome must say great ben affleck quite good rosamund pike great whole plot keeps guessing edge seat overall great movie highly recommend great watch significant much families


In [28]:
tokenizer = Tokenizer()
train_rev_tokens = tokenizer.texts_to_sequences(words_prep)

In [36]:
# words_prep

In [37]:
# train_rev_tokens

In [31]:
train_rev_pad = pad_sequences(train_rev_tokens, maxlen=10000)

In [38]:
# train_rev_pad

In [34]:
model_exp.predict(train_rev_pad)

2/2 [==============================] - 0s 6ms/step


array([[0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601061],
       [0.7601061],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],


In [103]:
# words

In [133]:
# words_idx[0]

In [106]:
# # words_ready = [el for el in words_idx if el < 10000]
# def asdfgh(words_idx):
#     n = 10000
#     words_ready = [0]*n
#     for i in range(n):
#         if i in words_idx and i < n:
#             words_ready[i] = 1
#     return words_ready

In [134]:
# words_idx

#### Positive review

In [45]:
text_string = str(input('Input your movie review: '))
words = preproc(text_string.lower())
# print(words)
words_prep = words.copy()
words_idx = get_index(words)

Input your movie review: Cunning. Clever. Sharp. Twisted. Dark. The way thrillers should be made. It is surprisingly good. Loved every minute of this masterpiece. These types of movies come once in a while and should be treasured immensely. Here's a mainstream movie that's actually REALLY good!


In [46]:
tokenizer = Tokenizer()
train_rev_tokens = tokenizer.texts_to_sequences(words_prep)

In [47]:
train_rev_pad = pad_sequences(train_rev_tokens, maxlen=10000)

In [50]:
model_exp.predict(train_rev_pad)

1/1 [==============================] - 0s 22ms/step


array([[0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062],
       [0.7601062]], dtype=float32)

In [109]:
# word_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [239]:
# def to_vec(words):
#     vecs = []
#     for word in words:
#         if word in word_vectors:
#             vctr = word_vectors.get_vector(word, norm=False)
#             vecs.append(vctr)
# #     vctr.tolist()
#     return np.asarray(vctr)

In [330]:
# words_vecs = to_vec(words)
# vectorizer = TfidfVectorizer()
# words_vecs = vectorizer.fit_transform(words)
# words_vecs = words_vecs.toarray()

Input your movie review: Thought this would be a good thriller, but ultimately it was draining and depressing. Would think any man who sees this film would never want to interact with any woman again...... it portrayed women as being monstrous and men as passive, crazy zombies. The story had so many loose ends and unbelievable scenes, such as where she returns home covered in blood..... then she's at the hospital - there had to be an exam, then giving an interview to detectives, then comes home still covered in blood....how much time elapsed during these scenes, and who would allow her to drive around, carry on, covered in blood for all that time? With great emphasis on crass words for women, violent scenes, and a score that sounded like a synthesizer importing screeches from an airport runway, this was one depressing film. The only place it was believable was in its depiction of the media and how they revel in tragedy and perversion. If this becomes our prime form of entertainment we 

In [222]:
# text_string = list(input('Input your movie review: '))
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(text_string)

In [444]:
# res_data = tf.expand_dims(train_data, axis=0)
# res_data

In [237]:
# vectorizer = TfidfVectorizer()
# words_vecs = vectorizer.fit_transform(words)
# words_vecs = words_vecs.toarray()
# vectorizer.get_feature_names_out()

In [236]:
# data_train = sequence.pad_sequences(words_vecs, maxlen=10000)